<a href="https://colab.research.google.com/github/hadi-ansari/MLOps/blob/main/TFX/Notebooks/0)_dogs_vs_cats_classification_without_pipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# A tutorial for training a model using cats & dogs dataset

In this notebook-based tutorial, we will create and run a ML model for a cats vs dogs classification. This model is not implemented inside a TFX pipeline and is a standalone code.

In [ ]:
import tensorflow as tf
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image
import pathlib

## Load you kaggle api key

In [ ]:
from google.colab import files
files.upload()

## Download the dataset
This will the install kaggle cli and download the dataset from kaggle.

In [ ]:
!pip install -q kaggle
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
'chmod 600 /root/.kaggle/kaggle.json'
!kaggle datasets download -d salader/dogs-vs-cats
!unzip dogs-vs-cats.zip
!rm -rf test train dogs-vs-cats.zip

## Run this reduce the size of dataset
It might take a very long time to train the model or troubleshoot the code if the size of the dataset is too big.

In [ ]:
!ls -1 dogs_vs_cats/train/dogs/* | tail -n +501 | xargs rm 
!ls -1 dogs_vs_cats/train/cats/* | tail -n +501 | xargs rm

!ls -1 dogs_vs_cats/test/dogs/* | tail -n +151 | xargs rm 
!ls -1 dogs_vs_cats/test/cats/* | tail -n +151 | xargs rm 

## Variable definition

In [ ]:
batch_size = 128
epochs = 3
IMG_HEIGHT = 150
IMG_WIDTH = 150

### Experiment
This is just to test how to load an image and resize it. It has nothing to do with the training code.

In [ ]:
import cv2
import matplotlib.pyplot as plt

temp_image = cv2.imread("/content/dogs_vs_cats/train/cats/cat.10003.jpg")
temp_image = cv2.cvtColor(temp_image, cv2.COLOR_BGR2RGB)
temp_image = temp_image/255.0
temp_gray_image = tf.image.rgb_to_grayscale(temp_image)
temp_gray_image = tf.squeeze(temp_gray_image, axis=-1)
temp_image = tf.image.resize_with_pad(temp_image, target_height=100, target_width=100)
print(temp_image.shape)

plt.figure()
plt.imshow(temp_image)
plt.colorbar()
plt.grid(False)
plt.show()

## Convert image dataset to TFRecords

We followed the instructions according [this](https://ai.plainenglish.io/a-quick-and-simple-guide-to-tfrecord-c421337a6562).


In [ ]:
import tensorflow as tf
import numpy as np
import os
from PIL import Image
import random
import cv2
import matplotlib.pyplot as plt

data_root="dogs_vs_cats"

# Setup the train and test imgage directories
train_dir = os.path.join(data_root, "train")
test_dir = os.path.join(data_root, "test")

# setup train and test TFRecord files
train_tfrecord='train_data.tfrecords'
test_tfrecord = 'test_data.tfrecords'

# Define the name of folders of each class
# We only have two classes in this case.
folders=['dogs', 'cats']

# List all train and test image path
train_image_path=[]
test_image_path=[]

for i in range(len(folders)):
    for file in os.listdir(os.path.join(train_dir, folders[i])):
        train_image_path.append(os.path.join(train_dir, folders[i], file))
    for file in os.listdir(os.path.join(test_dir, folders[i])):
        test_image_path.append( os.path.join(test_dir, folders[i], file))


print("Number of train images found: ", len(train_image_path))
print("Number of test images found: ", len(test_image_path))

# Shuffle the image paths for better accuracy and precision
random.seed(0)
random.shuffle(train_image_path)
random.shuffle(test_image_path)

# create train and test lables for shuffled image paths
# 0 for cat and 1 for dog
train_labels=[]
test_labels=[]
for i in range(len(train_image_path)):
    if os.path.basename(train_image_path[i])[:3]=='cat':
        train_labels.append(0)
    else:
        train_labels.append(1)

for i in range(len(test_image_path)):
    if os.path.basename(test_image_path[i])[:3]=='cat':
        test_labels.append(0)
    else:
        test_labels.append(1)

# The following functions can be used to convert a value to a type compatible
# with tf.train.Example.

def _bytes_feature(value):
  """Returns a bytes_list from a string / byte."""
  if isinstance(value, type(tf.constant(0))):
    value = value.numpy() # BytesList won't unpack a string from an EagerTensor.
  return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))

def _float_feature(value):
  """Returns a float_list from a float / double."""
  return tf.train.Feature(float_list=tf.train.FloatList(value=[value]))

def _int64_feature(value):
  """Returns an int64_list from a bool / enum / int / uint."""
  return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))

def serialize_example(image_string, label):
    ## Create a dictionary with features for images and their target labels
    image_shape = tf.io.decode_jpeg(image_string).shape

    feature = {
      'label': _int64_feature(label),
      'image_raw': _bytes_feature(image_string),
    }
    #  Create a Features message using tf.train.Example.
    example_proto = tf.train.Example(features=tf.train.Features(feature=feature))
    #serializes the message and returns it as a string. Note that the bytes are binary
    return example_proto.SerializeToString()

def write_TFRecord(image_path, label):
    image_string = open(image_path, 'rb').read()
    example = serialize_example(image_string, label)
    return example

#Write Train TFRecord file
with tf.io.TFRecordWriter(train_tfrecord) as writer:
    for image_path, label in zip(train_image_path, train_labels):
        writer.write(write_TFRecord(image_path, int(label)))

#Write Test TFRecord file
with tf.io.TFRecordWriter(test_tfrecord) as writer:
    for image_path, label in zip(test_image_path, test_labels):
         writer.write(write_TFRecord(image_path, int(label)))

## Restructure the files
This commands restructure the files and create a directory named `dataset` and move the TFRecord files into `dataset/train` and `dataset/test`.

In [ ]:
!rm -rf dataset
!mkdir dataset
!mkdir dataset/train
!mkdir dataset/test
!mv train_data.tfrecords dataset/train
!mv test_data.tfrecords dataset/test

## The old way of loading the dataset
Run this if you want to load the dataset from directories in **JPEG** format.

In [ ]:
train_ds = tf.keras.utils.image_dataset_from_directory(
    "./dogs_vs_cats/train",
    labels="inferred",
    label_mode="int",
    batch_size=32,
    image_size=(IMG_HEIGHT, IMG_WIDTH),
    shuffle=True,
)

eval_ds = tf.keras.utils.image_dataset_from_directory(
    "./dogs_vs_cats/test",
    labels="inferred",
    label_mode="int",
    batch_size=32,
    image_size=(IMG_HEIGHT, IMG_WIDTH),
    shuffle=True,
)


## The newer way of loading the dataset
Run this if you want to load the dataset stored in **TFRecord** format.

In [ ]:
# Define the feature description of your tfrecord dataset
feature_description = {
    'label': tf.io.FixedLenFeature([], tf.int64),
    'image_raw': tf.io.FixedLenFeature([], tf.string),
}
# Define a function to parse the tfrecord dataset
def _parse_function_old(example_proto):
    # Parse the input `tf.train.Example` proto using the feature description
    features = tf.io.parse_single_example(example_proto, feature_description)
    return features

# Define a function to parse the tfrecord dataset
def _parse_function_new(example_proto):
    # Parse the input `tf.train.Example` proto using the feature description
    features = tf.io.parse_single_example(example_proto, feature_description)
    
    # Decode the raw image data
    image = tf.io.decode_jpeg(features['image_raw'], channels=3)
    # Resize the image
    image = tf.image.resize(image, [150, 150])
    
    # Normalize the pixel values
    # image = tf.cast(image, tf.float32) / 255.0
    
    # Get the label
    label = features['label']
    
    return image, label

# Load the train tfrecord dataset
train_ds = tf.data.TFRecordDataset('./dataset/train/train_data.tfrecords')
train_ds = train_ds.map(_parse_function_new)
train_ds = train_ds.batch(batch_size)

# Load the test tfrecord dataset
eval_ds = tf.data.TFRecordDataset('./dataset/test/test_data.tfrecords')
eval_ds = eval_ds.map(_parse_function_new)
eval_ds = eval_ds.batch(batch_size)


## Write model training code
We will create a simple CNN (Convolutional Neural Network) model for cats vs dogs classification using TensorFlow Keras
API. This model training code will be saved to a separate file.

In [ ]:
import tensorflow as tf
import matplotlib.pyplot as plt
import os
import gzip

# Create the model
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, 3, padding='same', activation='relu', input_shape=(IMG_HEIGHT, IMG_WIDTH, 3)),
    tf.keras.layers.Conv2D(32, 3, padding='same', activation='relu'),
    tf.keras.layers.MaxPooling2D(),

    tf.keras.layers.Conv2D(64, 3, padding='same', activation='relu'),
    tf.keras.layers.Conv2D(64, 3, padding='same', activation='relu'),
    tf.keras.layers.MaxPooling2D(),

    tf.keras.layers.Conv2D(128, 3, padding='same', activation='relu'),
    tf.keras.layers.Conv2D(128, 3, padding='same', activation='relu'),
    tf.keras.layers.MaxPooling2D(),

    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(1, activation='sigmoid')
])


# Compile the model
model.compile(optimizer='adam',
          loss=tf.keras.losses.BinaryCrossentropy(),
          metrics=['accuracy'])

# Print a summary of the model
model.summary()

# Train our model
history = model.fit(
    train_ds,
    epochs=epochs,
    validation_data=eval_ds)


acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(epochs)

plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

## Prediction

In [ ]:
probability_model = tf.keras.Sequential([model, 
                                         tf.keras.layers.Softmax()])


predictions = probability_model.predict(eval_ds)

In [ ]:
# Use the trained model to predict on the eval_ds dataset
predictions = model.predict(eval_ds)

# Print the predictions for the first 10 images in the dataset
for image, prediction in zip(eval_ds.take(10), predictions[:10]):
    predicted_label = "dog" if prediction > 0.5 else "cat"
    true_label = "dog" if image[1].numpy()[0] == 1 else "cat"
    print("Predicted label: ", predicted_label)
    print("True label: ", true_label)